In [1]:
from data_generator import *
from datetime import datetime


In [2]:
from polygon import StocksClient
client = StocksClient(polygon_token)
ticker = "ABT"

In [3]:
import yfinance as yf


In [101]:
from pricing_model import *
from torch import tensor

import pandas as pd
import numpy as np
import torch

def ticker_to_x(dates, ticker, market_index_data, treasury_rates, earnings_per_share, annual_dividend):
    # Fetch data using your client method
    data = client.get_aggregate_bars(ticker, '2023-04-11', '2024-04-12', multiplier=1, timespan='day')["results"]
    data_df = pd.DataFrame(data)
    # get row by dates
    # Assuming you have a DataFrame called 'data_df' with a 'date' column
    # and you want to get the row for a specific date '2023-04-11'


    data_df["t"] = pd.to_datetime(data_df['t'], unit='ms').dt.strftime("%Y-%m-%d")
    # print(data_df)
    # # Join market index data and treasury rates on timestamps
    # data_df['market_returns'] = market_index_data['market_returns']
    # data_df['risk_free_rate'] = treasury_rates['6 Mo']
    # Assuming you have a DataFrame called 'data_df' with a 'date' column
    # dates = ['2023-04-11', '2023-04-12', '2023-04-13']

    # Filter rows based on the dates
    data_df = data_df.loc[data_df['t'].isin(dates)]

    # Join market index data and treasury rates on timestamps
    data_df['market_returns'] = market_index_data['market_returns']
    data_df['risk_free_rate'] = treasury_rates['6 Mo']
    
    # Prepare a list to hold tensor data
    tensor_data = []
    returns = data_df['c'].pct_change().dropna()
    # Loop over dataframe to compute features per day
    # print(returns)
    covariance = data_df['market_returns'].cov(data_df['market_returns'])
    market_variance = data_df['market_returns'].var()
    beta = covariance / market_variance if market_variance != 0 else 0
    alpha = data_df['market_returns'].mean() - (data_df['risk_free_rate'] + beta * (data_df['market_returns'].mean() - data_df['risk_free_rate']))

    # Calculate Sharpe Ratio
    excess_returns = data_df['market_returns'] - data_df['risk_free_rate']
    sharpe_ratio = excess_returns.mean() / excess_returns.std() if excess_returns.std() != 0 else 0
    
    # Calculate Volatility
    volatility = data_df['market_returns'].std()
    pe_ratio = data_df['c'].mean() / earnings_per_share
    
    dividend_yield = annual_dividend / data_df['c'].mean()
    print(type(dividend_yield))
    return torch.tensor([beta, alpha, sharpe_ratio, volatility, pe_ratio, dividend_yield])


# the function to import the parameters for a company from param_data by importing from corresponding "ticker".csv file as time series of torch tensors
def import_params(ticker):
	# read the csv file
    df = pd.read_csv('param_data/' + ticker + '.csv')
    dates = df['date']
    columns = df.columns[2:]
	# turn rows into tensors and append them to the the tensor
    params = [tensor(df[col].values) for col in columns]
    params = torch.stack(params, dim=1)
    
    return params, dates

In [102]:
from polygon import ReferenceClient
ref_client= ReferenceClient(polygon_token)

In [103]:
def get_market_data(ticker):
    # Download historical data for the specified ticker between the given dates
    data = yf.download(ticker, start="2023-04-10", end="2024-04-13")
    # Calculate daily returns and treat these as 'market returns'
    data['market_returns'] = data['Close'].pct_change()
    data = data.dropna()
    return data

market_index_data = get_market_data('^GSPC')
market_index_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,market_returns
Date,,,,,,,
2023-04-11,4110.290039,4124.259766,4102.609863,4108.939941,4108.939941,3665830000,-0.000041
2023-04-12,4121.720215,4134.370117,4086.939941,4091.949951,4091.949951,3633120000,-0.004135
2023-04-13,4100.040039,4150.259766,4099.399902,4146.220215,4146.220215,3596590000,0.013263
2023-04-14,4140.109863,4163.189941,4113.200195,4137.640137,4137.640137,3575690000,-0.002069
2023-04-17,4137.169922,4151.720215,4123.180176,4151.319824,4151.319824,3611180000,0.003306
...,...,...,...,...,...,...,...
2024-04-08,5211.370117,5219.569824,5197.350098,5202.390137,5202.390137,3278180000,-0.000375
2024-04-09,5217.029785,5224.810059,5160.779785,5209.910156,5209.910156,3400680000,0.001445
2024-04-10,5167.879883,5178.430176,5138.700195,5160.640137,5160.640137,3845930000,-0.009457


In [104]:
def fetch_eps(ticker):
    stock = yf.Ticker(ticker)
    eps = stock.info['trailingEps']  # Trailing twelve months EPS
    return eps

In [105]:
# Example usage
# Assuming you have provided appropriate data for market_index_data, treasury_rates, etc.
earnings_per_share = fetch_eps(ticker)  # Example EPS value
y, dates = import_params(ticker)
treasury_rates = pd.read_csv("five-year-rates.csv").get(["Date", "6 Mo"])
annual_dividend = sum([res["cash_amount"] for res in  ref_client.get_stock_dividends(ticker, pay_date_gte="2023-04-11", pay_date_lte="2024-04-12", dividend_type="CD")["results"]])
print(annual_dividend)
tensor_features = ticker_to_x(dates, 'AAPL', market_index_data, treasury_rates, earnings_per_share, annual_dividend)
# tensor_features

2.08
<class 'numpy.float64'>


TypeError: cannot convert the series to <class 'float'>

In [ ]:
# generate size 7 sliding windows of the parameters
x = tensor_features.nan_to_num(0)
y = import_params(ticker)
# (Joy) only trained one stock's data?